In [46]:
import pandas as pd
from gurobipy import GRB
import gurobipy as gb

In [47]:
# Load the data from the Excel files using pandas
centers_df = pd.read_csv("C:/Users/tauny/Desktop/Schulich - School/Classes/Winter 2025 - Semester 3/Models and Apps in Operational Research/Ass/Ass 1/centers.csv")  # Data for centers
processing_df = pd.read_csv("C:/Users/tauny/Desktop/Schulich - School/Classes/Winter 2025 - Semester 3/Models and Apps in Operational Research/Ass/Ass 1/processing.csv")  # Data for processing facilities
farms_df = pd.read_csv("C:/Users/tauny/Desktop/Schulich - School/Classes/Winter 2025 - Semester 3/Models and Apps in Operational Research/Ass/Ass 1/farms.csv")  # Data for famrms

In [48]:
# Check the first few rows of each DataFrame
print(farms_df.head())
print(processing_df.head())
print(centers_df.head())

  Farm_ID  Bio_Material_Capacity_Tons  Quality  Cost_Per_Ton  \
0  Farm_1                         478        2        127.46   
1  Farm_2                         308        2        137.42   
2  Farm_3                         516        3        189.20   
3  Farm_4                         367        1         66.23   
4  Farm_5                         499        1         86.06   

   Transport_Cost_To_Plant_1  Transport_Cost_To_Plant_2  \
0                   2.055668                   1.803083   
1                   2.575862                   1.600978   
2                   2.680801                   2.606527   
3                   2.296063                   1.544005   
4                   1.677500                   2.445887   

   Transport_Cost_To_Plant_3  Transport_Cost_To_Plant_4  \
0                   1.069480                   2.714386   
1                   1.639759                   1.096792   
2                   2.193316                   1.297035   
3                   2.20

In [49]:
# Get the number of farms, processing plants, and home centers
num_farms = len(farms_df)
num_processing_plants = len(processing_df)
num_home_centers = len(centers_df)

In [50]:
# Create a new optimization modelto minimize cost
model = gb.Model("Ass 1")

In [51]:
# Decision variables

# x[i, j] = amount of raw material from farm i to processing facility j
# y[j, k] = amount of fertilizer from processing facility j to home center k
x = model.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")

In [52]:
# Print the column names to verify
print(farms_df.columns)
print(processing_df.columns)

Index(['Farm_ID', 'Bio_Material_Capacity_Tons', 'Quality', 'Cost_Per_Ton',
       'Transport_Cost_To_Plant_1', 'Transport_Cost_To_Plant_2',
       'Transport_Cost_To_Plant_3', 'Transport_Cost_To_Plant_4',
       'Transport_Cost_To_Plant_5', 'Transport_Cost_To_Plant_6',
       'Transport_Cost_To_Plant_7', 'Transport_Cost_To_Plant_8',
       'Transport_Cost_To_Plant_9', 'Transport_Cost_To_Plant_10',
       'Transport_Cost_To_Plant_11', 'Transport_Cost_To_Plant_12',
       'Transport_Cost_To_Plant_13', 'Transport_Cost_To_Plant_14',
       'Transport_Cost_To_Plant_15', 'Transport_Cost_To_Plant_16',
       'Transport_Cost_To_Plant_17', 'Transport_Cost_To_Plant_18'],
      dtype='object')
Index(['Processing_Plant_ID', 'Region', 'Capacity_Tons',
       'Processing_Cost_Per_Ton', 'Transport_Cost_To_Center_1',
       'Transport_Cost_To_Center_2', 'Transport_Cost_To_Center_3',
       'Transport_Cost_To_Center_4', 'Transport_Cost_To_Center_5',
       'Transport_Cost_To_Center_6',
       ...
     

In [53]:
# Objective function: Minimize all relevant costs
model.setObjective(
    # 1. Transportation cost for raw material (farm to processing plant)
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 2. Procurement cost for raw material (cost per ton at farm)
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 3. Processing cost per ton (cost of processing raw material)
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 4. Transportation cost for fertilizer (processing plant to home center)
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    
    gb.GRB.MINIMIZE
)

In [54]:
# Constraints
# Land capacity constraint: total raw material sent to processing facilities should not exceed farm capacities
land_constraint = model.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

# Processing capacity constraint: total raw material received at processing facilities should not exceed processing capacity
processing_constraint = model.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

# Home center demand constraint: total fertilizer sent to home centers should meet their demand
demand_constraint = model.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Link fertilizer delivery to raw material processed
fertilizer_constraint = model.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) >= y[j, k] for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="Fertilizer_Processing_Limitation"
)

In [55]:
# Solve the model
model.optimize()

# Print the optimal solution
print("The optimal solution: ", model.objVal)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2205 rows, 6318 columns and 469800 nonzeros
Model fingerprint: 0xbb188d48
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.12s
Presolved: 2205 rows, 6318 columns, 469800 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.05s

Barrier performed 0 iterations in 0.32 seconds (0.23 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1867    1.0641657e+05   0.000000e+00   0.000000e+00      0s

Solved in 1867 iterations and 0.33 secon

The minimal cost that can be achieved is $106,416.57

In [56]:
# Check if the optimization was successful
if model.status == GRB.OPTIMAL:
    # Print the optimal amount of raw material transported
    print("Optimal Amount Transported (Raw Material):")
    for i in range(num_farms):
        for j in range(num_processing_plants):
            print(f"Raw Material from Farm {i+1} to Facility {j+1}: {x[i, j].x} tons")

    # Print the optimal amount of fertilizer transported
    print("Optimal Amount Transported (Fertilizer):")
    for j in range(num_processing_plants):
        for k in range(num_home_centers):
            print(f"Fertilizer from Facility {j+1} to Home Center {k+1}: {y[j, k].x} tons")
else:
    print("Optimization was not successful.")

# Print sensitivity information for constraints
for i in range(num_farms):
    print(f"Sensitivity Information for Farm Capacity Constraint {i+1}:")
    print(f"Shadow Price: {land_constraint[i].pi}")
    print(f"Range of Feasibility: {land_constraint[i].SARHSUp}, {land_constraint[i].SARHSLow}")

Optimal Amount Transported (Raw Material):
Raw Material from Farm 1 to Facility 1: 0.0 tons
Raw Material from Farm 1 to Facility 2: 0.0 tons
Raw Material from Farm 1 to Facility 3: 0.0 tons
Raw Material from Farm 1 to Facility 4: 0.0 tons
Raw Material from Farm 1 to Facility 5: 0.0 tons
Raw Material from Farm 1 to Facility 6: 0.0 tons
Raw Material from Farm 1 to Facility 7: 0.0 tons
Raw Material from Farm 1 to Facility 8: 0.0 tons
Raw Material from Farm 1 to Facility 9: 0.0 tons
Raw Material from Farm 1 to Facility 10: 0.0 tons
Raw Material from Farm 1 to Facility 11: 0.0 tons
Raw Material from Farm 1 to Facility 12: 0.0 tons
Raw Material from Farm 1 to Facility 13: 0.0 tons
Raw Material from Farm 1 to Facility 14: 0.0 tons
Raw Material from Farm 1 to Facility 15: 0.0 tons
Raw Material from Farm 1 to Facility 16: 0.0 tons
Raw Material from Farm 1 to Facility 17: 0.0 tons
Raw Material from Farm 1 to Facility 18: 0.0 tons
Raw Material from Farm 2 to Facility 1: 0.0 tons
Raw Material from

In [57]:
# Print the optimal amount of raw material transported only if non-zero
print("Optimal Amount Transported (Raw Material):")
for i in range(num_farms):
    for j in range(num_processing_plants):
        if x[i, j].x > 0:  # Only print non-zero values
            print(f"Raw Material from Farm {i+1} to Facility {j+1}: {x[i, j].x} tons")

Optimal Amount Transported (Raw Material):
Raw Material from Farm 135 to Facility 7: 55.28571428571429 tons
Raw Material from Farm 135 to Facility 8: 12.285714285714278 tons
Raw Material from Farm 135 to Facility 10: 21.85714285714289 tons
Raw Material from Farm 135 to Facility 14: 43.85714285714289 tons
Raw Material from Farm 135 to Facility 15: 26.0 tons
Raw Material from Farm 135 to Facility 16: 56.99999999999997 tons
Raw Material from Farm 150 to Facility 4: 24.71428571428575 tons
Raw Material from Farm 244 to Facility 2: 17.14285714285714 tons
Raw Material from Farm 244 to Facility 4: 109.8571428571428 tons
Raw Material from Farm 244 to Facility 6: 21.85714285714289 tons
Raw Material from Farm 244 to Facility 9: 74.71428571428571 tons
Raw Material from Farm 244 to Facility 12: 31.42857142857143 tons


In [58]:
# Print the optimal amount of fertilizer transported only if non-zero
print("Optimal Amount Transported (Fertilizer):")
for j in range(num_processing_plants):
    for k in range(num_home_centers):
        if y[j, k].x > 0:  # Only print non-zero values
            print(f"Fertilizer from Facility {j+1} to Home Center {k+1}: {y[j, k].x} tons")

Optimal Amount Transported (Fertilizer):
Fertilizer from Facility 2 to Home Center 1: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 2: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 4: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 5: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 7: 1.1428571428571246 tons
Fertilizer from Facility 2 to Home Center 10: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 11: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 12: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 16: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 18: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 19: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 21: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 22: 17.14285714285714 tons
Fertilizer from Facility 2 to Home Center 23: 17.14285714285714 to

In [59]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

Number of Decision Variables:  6318
Total Transportation cost:  106416.57324184533

    Variable            X 
-------------------------
RawMaterial[134,6]      55.2857 
RawMaterial[134,7]      12.2857 
RawMaterial[134,9]      21.8571 
RawMaterial[134,13]      43.8571 
RawMaterial[134,14]           26 
RawMaterial[134,15]           57 
RawMaterial[149,3]      24.7143 
RawMaterial[243,1]      17.1429 
RawMaterial[243,3]      109.857 
RawMaterial[243,5]      21.8571 
RawMaterial[243,8]      74.7143 
RawMaterial[243,11]      31.4286 
Fertilizer[1,0]      17.1429 
Fertilizer[1,1]      17.1429 
Fertilizer[1,3]      17.1429 
Fertilizer[1,4]      17.1429 
Fertilizer[1,6]      1.14286 
Fertilizer[1,9]      17.1429 
Fertilizer[1,10]      17.1429 
Fertilizer[1,11]      17.1429 
Fertilizer[1,15]      17.1429 
Fertilizer[1,17]      17.1429 
Fertilizer[1,18]      17.1429 
Fertilizer[1,20]      17.1429 
Fertilizer[1,21]      17.1429 
Fertilizer[1,22]      17.1429 
Fertilizer[1,23]      17.1429 
Fert

In [60]:
# Ensure fertilizer demand at home centers is met
print("Verifying Fertilizer Delivery to Home Centers:")
for k in range(num_home_centers):
    # Sum up the total fertilizer delivered to home center k
    total_fertilizer_delivered = sum(y[j, k].x for j in range(num_processing_plants))
    
    # Get the demand for fertilizer at home center k
    demand = centers_df.iloc[k]['Requested_Demand_Tons']
    
    # Print whether the demand is met
    if total_fertilizer_delivered == demand:
        print(f"Home Center {k+1}: Demand of {demand} tons met with {total_fertilizer_delivered} tons delivered.")
    else:
        print(f"Home Center {k+1}: Demand of {demand} tons NOT met. {total_fertilizer_delivered} tons delivered.")

Verifying Fertilizer Delivery to Home Centers:
Home Center 1: Demand of 82 tons met with 82.0 tons delivered.
Home Center 2: Demand of 348 tons met with 348.0 tons delivered.
Home Center 3: Demand of 464 tons met with 464.0 tons delivered.
Home Center 4: Demand of 161 tons met with 161.0 tons delivered.
Home Center 5: Demand of 340 tons met with 340.0 tons delivered.
Home Center 6: Demand of 86 tons met with 86.0 tons delivered.
Home Center 7: Demand of 480 tons met with 480.0 tons delivered.
Home Center 8: Demand of 149 tons met with 149.0 tons delivered.
Home Center 9: Demand of 72 tons met with 72.0 tons delivered.
Home Center 10: Demand of 469 tons NOT met. 468.99999999999994 tons delivered.
Home Center 11: Demand of 473 tons NOT met. 472.99999999999994 tons delivered.
Home Center 12: Demand of 267 tons met with 267.0 tons delivered.
Home Center 13: Demand of 138 tons met with 138.0 tons delivered.
Home Center 14: Demand of 258 tons met with 258.0 tons delivered.
Home Center 15: De

In [61]:
# Initialize lists to hold the total transportation costs for farm-to-processing routes
raw_material_costs = []

# Calculate total costs for each farm-to-processing route
for i in range(num_farms):
    for j in range(num_processing_plants):
        transport_cost = farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}']  # Get transportation cost for that route
        amount_transported = x[i, j].x  # Amount of raw material transported
        total_cost = transport_cost * amount_transported  # Total cost for this route
        raw_material_costs.append(total_cost)

# Calculate high, low, and average values for farm-to-processing routes
high_cost = max(raw_material_costs)
low_cost = min(raw_material_costs)
average_cost = sum(raw_material_costs) / len(raw_material_costs)

print(f"Raw Material Transportation Cost Stats:")
print(f"High Cost: {high_cost}")
print(f"Low Cost: {low_cost}")
print(f"Average Cost: {average_cost}")

Raw Material Transportation Cost Stats:
High Cost: 185.68901595923094
Low Cost: 0.0
Average Cost: 0.17970236402382092


Use these vals to set threshold.

In [62]:
# Set threshold to 1.5 times the average cost
threshold = 1.5 * average_cost 

# Initialize a list to store the routes and their total costs
high_cost_routes = []

# Reviewing high transportation cost routes (Farm to Processing Facility)
print("Reviewing Farm to Processing Plant Transportation Costs:")
for i in range(num_farms):
    for j in range(num_processing_plants):
        transport_cost = farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}']  # Get transportation cost for that route
        amount_transported = x[i, j].x  # Amount of raw material transported
        total_cost = transport_cost * amount_transported  # Total cost for this route
        
        if total_cost > threshold:  # threshold is a value you can set to find high-cost routes
            high_cost_routes.append((i+1, j+1, total_cost))  # Storing farm, processing plant, and total cost

# Sort the list of high-cost routes from high to low
high_cost_routes.sort(key=lambda x: x[2], reverse=True)

# Print the sorted high-cost routes
for route in high_cost_routes:
    print(f"High Cost Route: Farm {route[0]} to Processing Facility {route[1]}: {route[2]:.2f} units of cost.")

Reviewing Farm to Processing Plant Transportation Costs:
High Cost Route: Farm 244 to Processing Facility 9: 185.69 units of cost.
High Cost Route: Farm 244 to Processing Facility 4: 156.74 units of cost.
High Cost Route: Farm 135 to Processing Facility 7: 125.36 units of cost.
High Cost Route: Farm 135 to Processing Facility 14: 66.31 units of cost.
High Cost Route: Farm 135 to Processing Facility 16: 59.56 units of cost.
High Cost Route: Farm 244 to Processing Facility 12: 42.41 units of cost.
High Cost Route: Farm 135 to Processing Facility 15: 42.29 units of cost.
High Cost Route: Farm 135 to Processing Facility 10: 34.45 units of cost.
High Cost Route: Farm 244 to Processing Facility 6: 31.31 units of cost.
High Cost Route: Farm 150 to Processing Facility 4: 25.12 units of cost.
High Cost Route: Farm 244 to Processing Facility 2: 21.60 units of cost.
High Cost Route: Farm 135 to Processing Facility 8: 14.58 units of cost.


In [63]:
# Initialize lists to hold the total transportation costs for processing-to-home-center routes
fertilizer_costs = []

# Calculate total costs for each processing-to-home-center route
for j in range(num_processing_plants):
    for k in range(num_home_centers):
        transport_cost = processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}']  # Get transportation cost for that route
        amount_transported = y[j, k].x  # Amount of fertilizer transported
        total_cost = transport_cost * amount_transported  # Total cost for this route
        fertilizer_costs.append(total_cost)

# Calculate high, low, and average values for processing-to-home-center routes
high_cost_fertilizer = max(fertilizer_costs)
low_cost_fertilizer = min(fertilizer_costs)
average_cost_fertilizer = sum(fertilizer_costs) / len(fertilizer_costs)

print(f"Fertilizer Transportation Cost Stats:")
print(f"High Cost: {high_cost_fertilizer}")
print(f"Low Cost: {low_cost_fertilizer}")
print(f"Average Cost: {average_cost_fertilizer}")

Fertilizer Transportation Cost Stats:
High Cost: 489.7101667643006
Low Cost: 0.0
Average Cost: 42.14291432572718


In [64]:
# Set threshold to 1.5 times the average cost (or adjust this as needed)
threshold_fertilizer = 1.5 * average_cost_fertilizer  # You can also experiment with other multiples

# Initialize a list to store the routes and their total costs for fertilizer transportation
high_cost_fertilizer_routes = []

# Reviewing processing facility to home center transportation costs
print("Reviewing Processing Facility to Home Center Transportation Costs:")

for j in range(num_processing_plants):
    for k in range(num_home_centers):
        transport_cost = processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}']  # Get transportation cost for that route
        amount_transported = y[j, k].x  # Amount of fertilizer transported
        total_cost = transport_cost * amount_transported  # Total cost for this route
        
        # If the route exceeds the threshold, add it to the list
        if total_cost > threshold_fertilizer:
            high_cost_fertilizer_routes.append((j+1, k+1, total_cost))  # Storing processing plant, home center, and total cost

# Sort the list of high-cost fertilizer routes from high to low
high_cost_fertilizer_routes.sort(key=lambda x: x[2], reverse=True)

# Print the sorted high-cost fertilizer routes
for route in high_cost_fertilizer_routes:
    print(f"High Cost Route: Processing Facility {route[0]} to Home Center {route[1]}: {route[2]:.2f} units of cost.")

Reviewing Processing Facility to Home Center Transportation Costs:
High Cost Route: Processing Facility 4 to Home Center 41: 489.71 units of cost.
High Cost Route: Processing Facility 4 to Home Center 47: 465.27 units of cost.
High Cost Route: Processing Facility 4 to Home Center 61: 446.30 units of cost.
High Cost Route: Processing Facility 4 to Home Center 56: 430.96 units of cost.
High Cost Route: Processing Facility 4 to Home Center 50: 427.48 units of cost.
High Cost Route: Processing Facility 4 to Home Center 16: 425.58 units of cost.
High Cost Route: Processing Facility 4 to Home Center 96: 418.03 units of cost.
High Cost Route: Processing Facility 4 to Home Center 65: 413.46 units of cost.
High Cost Route: Processing Facility 4 to Home Center 54: 411.59 units of cost.
High Cost Route: Processing Facility 4 to Home Center 67: 404.68 units of cost.
High Cost Route: Processing Facility 4 to Home Center 34: 404.14 units of cost.
High Cost Route: Processing Facility 4 to Home Center

In [65]:
# Part C

# Create a new optimization modelto minimize cost
model_pc = gb.Model("Ass 1")

# Decision variables

# x[i, j] = amount of raw material from farm i to processing facility j
# y[j, k] = amount of fertilizer from processing facility j to home center k
x = model_pc.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pc.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")



# Objective function: Minimize all relevant costs
model_pc.setObjective(
    # 1. Transportation cost for raw material (farm to processing plant)
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 2. Procurement cost for raw material (cost per ton at farm)
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 3. Processing cost per ton (cost of processing raw material)
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 4. Transportation cost for fertilizer (processing plant to home center)
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    
    gb.GRB.MINIMIZE
)


# Constraints
# Land capacity constraint: total raw material sent to processing facilities should not exceed farm capacities
land_constraint = model_pc.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

# Processing capacity constraint: total raw material received at processing facilities should not exceed processing capacity
processing_constraint = model_pc.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

# Home center demand constraint: total fertilizer sent to home centers should meet their demand
demand_constraint = model_pc.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Link fertilizer delivery to raw material processed
fertilizer_constraint = model_pc.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) >= y[j, k] for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="Fertilizer_Processing_Limitation"
)

# Add the regional constraint to the new model (corrected to use y_part_c)
for j in range(num_processing_plants):
    for k in range(num_home_centers):
        if processing_df.iloc[j]["Region"] != centers_df.iloc[k]["Region"]:
            model_pc.addConstr(y[j, k] == 0)  # No fertilizer can be transported to this home center if the regions don't match

# Solve the model
model_pc.optimize()

# Print the optimal solution
print("The optimal solution: ", model_pc.objVal)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3579 rows, 6318 columns and 471174 nonzeros
Model fingerprint: 0xda8db569
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 2748 rows and 1374 columns
Presolve time: 0.05s
Presolved: 831 rows, 4944 columns, 124926 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier performed 0 iterations in 0.08 seconds (0.08 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     420    1.9568331e+05   0.000000e+00   0.000000e+00      0

The optimal cost when restricted to region is $195,683.31.

In [66]:
# Part D


# Create a new optimization modelto minimize cost
model_pd = gb.Model("Ass 1")

# Decision variables

# x[i, j] = amount of raw material from farm i to processing facility j
# y[j, k] = amount of fertilizer from processing facility j to home center k
x = model_pd.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pd.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")



# Objective function: Minimize all relevant costs
model_pd.setObjective(
    # 1. Transportation cost for raw material (farm to processing plant)
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 2. Procurement cost for raw material (cost per ton at farm)
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 3. Processing cost per ton (cost of processing raw material)
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    
    # 4. Transportation cost for fertilizer (processing plant to home center)
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    
    gb.GRB.MINIMIZE
)


# Constraints
# Land capacity constraint: total raw material sent to processing facilities should not exceed farm capacities
land_constraint = model_pd.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

# Processing capacity constraint: total raw material received at processing facilities should not exceed processing capacity
processing_constraint = model_pd.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

# Home center demand constraint: total fertilizer sent to home centers should meet their demand
demand_constraint = model_pd.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Link fertilizer delivery to raw material processed
fertilizer_constraint = model_pd.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) >= y[j, k] for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="Fertilizer_Processing_Limitation"
)

# Add the high-quality raw material constraint (only farms with quality 3 or 4 can be used)
for i in range(num_farms):
    if farms_df.iloc[i]["Quality"] < 3:
        # Set the transportation of raw material from this farm to any processing plant to 0 (block it)
        for j in range(num_processing_plants):
            model_pd.addConstr(x[i, j] == 0)

# Solve the model
model_pd.optimize()

# Print the optimal solution
if model_pd.status == gb.GRB.OPTIMAL:
    print(f"The optimal solution with high-quality raw material: {model_pd.objVal}")
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5337 rows, 6318 columns and 472932 nonzeros
Model fingerprint: 0x7543c184
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 3306 rows and 3132 columns
Presolve time: 0.09s
Presolved: 2031 rows, 3186 columns, 144072 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.03s

Barrier statistics:
 AA' NZ     : 2.354e+05
 Factor NZ  : 4.403e+05 (roughly 6 MB of memory)
 Factor Ops : 1.007e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.27 seconds (0.11 work units)
Barrier solve interrupted - model solved by a

The optimal solution with high-quality raw material: $156,382.75.

In [67]:
# Part e


# Create a new optimization model to minimize cost
model_pe1 = gb.Model("Ass 1")

# Decision Variables
x = model_pe1.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pe1.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")

# Objective function: Minimize all relevant costs (same as before)
model_pe1.setObjective(
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    GRB.MINIMIZE
)

# Constraints: basic ones first
land_constraint = model_pe1.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

processing_constraint = model_pe1.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

demand_constraint = model_pe1.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Applying new constraints: Sourcing Risk Mitigation and Supply Risk Mitigation
total_raw_material = sum(farms_df['Bio_Material_Capacity_Tons'])  # Total raw material from all farms

sourcing_constraint = model_pe1.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= 0.03 * total_raw_material for j in range(num_processing_plants)),
    name="SourcingRisk_Constraint_ProcessingPlant"
)


# Solve the model with just these new constraints
model_pe1.optimize()

# Output the result with all constraints applied
if model_pe1.status == GRB.OPTIMAL:
    print(f"Optimal Transportation and Procurement Cost with limiting raw material to 3% Constraints: {model_pe1.objVal}")
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 6318 columns and 15282 nonzeros
Model fingerprint: 0x95e59ec0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 387 rows and 6318 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.1171568e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.117156825e+04
Optimal Transportation and Procurement Cost with limiting raw material to 3% Constraints: 61171.56825290918


Optimal Transportation and Procurement Cost with  limiting the raw material to 3% for each plant constraint: $61,171.57.


In [68]:
# Create a new optimization model to minimize cost
model_pe2 = gb.Model("Ass 1")

# Decision Variables
x = model_pe2.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pe2.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")

# Objective function: Minimize all relevant costs (same as before)
model_pe2.setObjective(
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    GRB.MINIMIZE
)

# Constraints: basic ones first
land_constraint = model_pe2.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

processing_constraint = model_pe2.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

demand_constraint = model_pe2.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

supply_constraint = model_pe2.addConstrs(
    (y[j, k] <= 0.50 * gb.quicksum(y[j, kk] for kk in range(num_home_centers))
     for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="SupplyRisk_Constraint_ProcessingPlant_HomeCenter"
)

# Solve the model with just these new constraints
model_pe2.optimize()

# Output the result with all constraints applied
if model_pe2.status == GRB.OPTIMAL:
    print(f"Optimal Transportation and Procurement Cost with limiting fert to max 50% Constraints: {model_pe2.objVal}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2205 rows, 6318 columns and 198072 nonzeros
Model fingerprint: 0x8000b508
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 267 rows and 4482 columns
Presolve time: 0.06s
Presolved: 1938 rows, 1836 columns, 189108 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...


Barrier performed 0 iterations in 0.08 seconds (0.07 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       1    6.1173815e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterat

Optimal Transportation and Procurement Cost with limit the fertilizer that can be sent to a home center to 50% of the total from a processing facility: $61,173.82.

In [69]:
# Part F

# Create a new optimization model to minimize cost
model_pf = gb.Model("Ass 1")

# Decision Variables
x = model_pf.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pf.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")

# Objective function: Minimize all relevant costs (same as before)
model_pf.setObjective(
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * farms_df.iloc[i]["Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(x[i, j] * processing_df.iloc[j]["Processing_Cost_Per_Ton"] for i in range(num_farms) for j in range(num_processing_plants)) +
    gb.quicksum(y[j, k] * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}'] for j in range(num_processing_plants) for k in range(num_home_centers)),
    GRB.MINIMIZE
)

# Constraints: basic ones first
land_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

processing_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

demand_constraint = model_pf.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Add the regional constraint to the new model (corrected to use y_part_c)
for j in range(num_processing_plants):
    for k in range(num_home_centers):
        if processing_df.iloc[j]["Region"] != centers_df.iloc[k]["Region"]:
            model_pf.addConstr(y[j, k] == 0)  # No fertilizer can be transported to this home center if the regions don't match

# Add the high-quality raw material constraint (only farms with quality 3 or 4 can be used)
for i in range(num_farms):
    if farms_df.iloc[i]["Quality"] < 3:
        # Set the transportation of raw material from this farm to any processing plant to 0 (block it)
        for j in range(num_processing_plants):
            model_pf.addConstr(x[i, j] == 0)
            
# Applying new constraints: Sourcing Risk Mitigation and Supply Risk Mitigation
total_raw_material = sum(farms_df['Bio_Material_Capacity_Tons'])  # Total raw material from all farms

sourcing_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= 0.03 * total_raw_material for j in range(num_processing_plants)),
    name="SourcingRisk_Constraint_ProcessingPlant"
)

supply_constraint = model_pf.addConstrs(
    (y[j, k] <= 0.50 * gb.quicksum(y[j, kk] for kk in range(num_home_centers))
     for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="SupplyRisk_Constraint_ProcessingPlant_HomeCenter"
)

# Solve the model with just these new constraints
model_pf.optimize()

# Output the result with all constraints applied
if model_pf.status == GRB.OPTIMAL:
    print(f"Optimal Transportation and Procurement Cost with All Constraints: {model_pf.objVal}")
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6729 rows, 6318 columns and 207060 nonzeros
Model fingerprint: 0x991856bb
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 6165 rows and 5856 columns
Presolve time: 0.02s
Presolved: 564 rows, 462 columns, 12828 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9340133e+04   3.100000e+01   0.000000e+00      0s
       3    6.9350230e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds (0.03 work units)
Optimal objective  6.935023035e+04
Optimal Transportation and Procurement Cost with All Constraints: 69350.23035317978


Optimal cost of all together: $69,350.23.

In [70]:
# Part g


In [72]:
model_pf = gb.Model("Ass 1")

# Decision Variables
x = model_pf.addVars(num_farms, num_processing_plants, lb=0, vtype=GRB.CONTINUOUS, name="RawMaterial")
y = model_pf.addVars(num_processing_plants, num_home_centers, lb=0, vtype=GRB.CONTINUOUS, name="Fertilizer")

# Objective function: Minimize all relevant costs (same as before)
model_pf.setObjective(
    gb.quicksum(x[i, j] * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}'] for i in range(num_farms) for j in range(num_processing_plants)),
    GRB.MINIMIZE
)

# Constraints: basic ones first
land_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for j in range(num_processing_plants)) <= farms_df.iloc[i]['Bio_Material_Capacity_Tons'] for i in range(num_farms)),
    "Farm Capacity"
)

processing_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= processing_df.iloc[j]['Capacity_Tons'] for j in range(num_processing_plants)),
    "Processing Capacity"
)

demand_constraint = model_pf.addConstrs(
    (gb.quicksum(y[j, k] for j in range(num_processing_plants)) == centers_df.iloc[k]['Requested_Demand_Tons'] for k in range(num_home_centers)),
    "Home Center Demand"
)

# Add the regional constraint to the new model (corrected to use y_part_c)
for j in range(num_processing_plants):
    for k in range(num_home_centers):
        if processing_df.iloc[j]["Region"] != centers_df.iloc[k]["Region"]:
            model_pf.addConstr(y[j, k] == 0)  # No fertilizer can be transported to this home center if the regions don't match

# Add the high-quality raw material constraint (only farms with quality 3 or 4 can be used)
for i in range(num_farms):
    if farms_df.iloc[i]["Quality"] < 3:
        # Set the transportation of raw material from this farm to any processing plant to 0 (block it)
        for j in range(num_processing_plants):
            model_pf.addConstr(x[i, j] == 0)
            
# Applying new constraints: Sourcing Risk Mitigation and Supply Risk Mitigation
total_raw_material = sum(farms_df['Bio_Material_Capacity_Tons'])  # Total raw material from all farms

sourcing_constraint = model_pf.addConstrs(
    (gb.quicksum(x[i, j] for i in range(num_farms)) <= 0.03 * total_raw_material for j in range(num_processing_plants)),
    name="SourcingRisk_Constraint_ProcessingPlant"
)

supply_constraint = model_pf.addConstrs(
    (y[j, k] <= 0.50 * gb.quicksum(y[j, kk] for kk in range(num_home_centers))
     for j in range(num_processing_plants) for k in range(num_home_centers)),
    name="SupplyRisk_Constraint_ProcessingPlant_HomeCenter"
)

# Solve the model with just these new constraints
model_pf.optimize()

# Output the result with all constraints applied
if model_pf.status == GRB.OPTIMAL:
    print(f"Optimal Transportation and Procurement Cost with All Constraints: {model_pf.objVal}")
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6729 rows, 6318 columns and 207060 nonzeros
Model fingerprint: 0x49a0dc63
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 6165 rows and 5856 columns
Presolve time: 0.05s
Presolved: 564 rows, 462 columns, 12828 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.08 seconds (0.03 work units)
Optimal objective  0.000000000e+00
Optimal Transportation and Procurement Cost with All Constraints: 0.0


In [74]:
print("Objective function components:")
for i in range(num_farms):
    for j in range(num_processing_plants):
        print(f"Transport Cost from Farm {i+1} to Plant {j+1}: {x[i, j].x * farms_df.iloc[i][f'Transport_Cost_To_Plant_{j+1}']}")

for i in range(num_farms):
    for j in range(num_processing_plants):
        print(f"Procurement Cost from Farm {i+1}: {x[i, j].x * farms_df.iloc[i]['Cost_Per_Ton']}")

for j in range(num_processing_plants):
    for k in range(num_home_centers):
        print(f"Transport Cost from Plant {j+1} to Center {k+1}: {y[j, k].x * processing_df.iloc[j][f'Transport_Cost_To_Center_{k+1}']}")

Objective function components:
Transport Cost from Farm 1 to Plant 1: 0.0
Transport Cost from Farm 1 to Plant 2: 0.0
Transport Cost from Farm 1 to Plant 3: 0.0
Transport Cost from Farm 1 to Plant 4: 0.0
Transport Cost from Farm 1 to Plant 5: 0.0
Transport Cost from Farm 1 to Plant 6: 0.0
Transport Cost from Farm 1 to Plant 7: 0.0
Transport Cost from Farm 1 to Plant 8: 0.0
Transport Cost from Farm 1 to Plant 9: 0.0
Transport Cost from Farm 1 to Plant 10: 0.0
Transport Cost from Farm 1 to Plant 11: 0.0
Transport Cost from Farm 1 to Plant 12: 0.0
Transport Cost from Farm 1 to Plant 13: 0.0
Transport Cost from Farm 1 to Plant 14: 0.0
Transport Cost from Farm 1 to Plant 15: 0.0
Transport Cost from Farm 1 to Plant 16: 0.0
Transport Cost from Farm 1 to Plant 17: 0.0
Transport Cost from Farm 1 to Plant 18: 0.0
Transport Cost from Farm 2 to Plant 1: 0.0
Transport Cost from Farm 2 to Plant 2: 0.0
Transport Cost from Farm 2 to Plant 3: 0.0
Transport Cost from Farm 2 to Plant 4: 0.0
Transport Cost

In [76]:
# Remove all constraints from the model
for constr in model_pe1.getConstrs():
    model_pe1.remove(constr)

# Solve the model with no constraints
model_pe1.optimize()

# Output the result with no constraints applied
if model_pe1.status == GRB.OPTIMAL:
    print(f"Optimal Cost without Constraints: {model_pe1.objVal}")
elif model_pe1.status == GRB.INFEASIBLE:
    print("The model is infeasible without constraints.")
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



Optimize a model with 0 rows, 6318 columns and 0 nonzeros
Model fingerprint: 0x2cf0415d
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 6318 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Optimal Cost without Constraints: 0.0
